# Implement NN with training in numpy
In this section we implement a 2-layer neural network in numpy
and write code to compute gradients and train it. In later sections
we will substitute this with pyTorch framework.

In [24]:
import numpy as np

# N is batch size; D_in is input dimension;
# H is hidden dimension; D_out is output dimension.
N, D_in, H, D_out = 64, 1000, 100, 10

x = np.random.randn(N, D_in)
y = np.random.randn(N, D_out)

w1 = np.random.randn(D_in, H)
w2 = np.random.randn(H, D_out)

learning_rate = 1e-6

for epoch in range(1, 101):
    # Forward pass: compute predicted y
    h = x.dot(w1)
    h_relu = np.maximum(h, 0)
    y_pred = h_relu.dot(w2)

    loss = np.square(y_pred - y).sum()
    if epoch % 10 == 0:
        print ("Loss after epoch#{} = {}".format(epoch, loss))

    # Backprop to compute gradients of w1 and w2 with respect to loss
    grad_y_pred = 2.0 * (y_pred - y)
    grad_w2 = h_relu.T.dot(grad_y_pred)
    grad_h_relu = grad_y_pred.dot(w2.T)
    grad_h = grad_h_relu.copy()
    grad_h[h < 0] = 0
    grad_w1 = x.T.dot(grad_h)

    # Update weights
    w1 -= learning_rate * grad_w1
    w2 -= learning_rate * grad_w2

Loss after epoch#10 = 3846733.26435
Loss after epoch#20 = 295808.615995
Loss after epoch#30 = 92206.8380658
Loss after epoch#40 = 35240.335839
Loss after epoch#50 = 14941.9370407
Loss after epoch#60 = 6800.10974393
Loss after epoch#70 = 3252.48935337
Loss after epoch#80 = 1618.7259482
Loss after epoch#90 = 833.413745886
Loss after epoch#100 = 441.501260191


# Raw NN code using torch
In this section, we write the same code but using pytorch. torch
tensors can easily be instantiated on cpu / gpu for speedup

In [27]:
# -*- coding: utf-8 -*-

import torch


dtype = torch.float
device = torch.device("cpu")
#device = torch.device("cuda:0") # Uncomment this to run on GPU

# N is batch size; D_in is input dimension;
# H is hidden dimension; D_out is output dimension.
N, D_in, H, D_out = 64, 1000, 100, 10

# Create random input and output data
x = torch.randn(N, D_in, device=device, dtype=dtype)
y = torch.randn(N, D_out, device=device, dtype=dtype)

# Randomly initialize weights
w1 = torch.randn(D_in, H, device=device, dtype=dtype)
w2 = torch.randn(H, D_out, device=device, dtype=dtype)

learning_rate = 1e-6
for epoch in range(1, 101):
    # Forward pass: compute predicted y
    h = x.mm(w1)
    h_relu = h.clamp(min=0)
    y_pred = h_relu.mm(w2)

    # Compute and print loss
    loss = (y_pred - y).pow(2).sum().item()
    if epoch % 10 == 0:
        print ("Loss after epoch#{} = {}".format(epoch, loss))


    # Backprop to compute gradients of w1 and w2 with respect to loss
    grad_y_pred = 2.0 * (y_pred - y)
    grad_w2 = h_relu.t().mm(grad_y_pred)
    grad_h_relu = grad_y_pred.mm(w2.t())
    grad_h = grad_h_relu.clone()
    grad_h[h < 0] = 0
    grad_w1 = x.t().mm(grad_h)

    # Update weights using gradient descent
    w1 -= learning_rate * grad_w1
    w2 -= learning_rate * grad_w2

Loss after epoch#10 = 1915199.0
Loss after epoch#20 = 233240.65625
Loss after epoch#30 = 73698.953125
Loss after epoch#40 = 27723.0371094
Loss after epoch#50 = 11382.8535156
Loss after epoch#60 = 4953.1875
Loss after epoch#70 = 2240.3671875
Loss after epoch#80 = 1042.73400879
Loss after epoch#90 = 496.761627197
Loss after epoch#100 = 242.443161011


# AutoGrad based training
Train a model using pyTorch but using autograd to avoid manual differentiation part

In [31]:
# -*- coding: utf-8 -*-
import torch

dtype = torch.float
device = torch.device("cpu")
# device = torch.device("cuda:0") # Uncomment this to run on GPU

# N is batch size; D_in is input dimension;
# H is hidden dimension; D_out is output dimension.
N, D_in, H, D_out = 64, 1000, 100, 10

# Create random Tensors to hold input and outputs.
# Setting requires_grad=False indicates that we do not need to compute gradients
# with respect to these Tensors during the backward pass.
x = torch.randn(N, D_in, device=device, dtype=dtype)
y = torch.randn(N, D_out, device=device, dtype=dtype)

# Create random Tensors for weights.
# Setting requires_grad=True indicates that we want to compute gradients with
# respect to these Tensors during the backward pass.
w1 = torch.randn(D_in, H, device=device, dtype=dtype, requires_grad=True)
w2 = torch.randn(H, D_out, device=device, dtype=dtype, requires_grad=True)

learning_rate = 1e-6
for epoch in range(1, 101):
    # Forward pass: compute predicted y using operations on Tensors; these
    # are exactly the same operations we used to compute the forward pass using
    # Tensors, but we do not need to keep references to intermediate values since
    # we are not implementing the backward pass by hand.
    y_pred = x.mm(w1).clamp(min=0).mm(w2)

    # Compute and print loss using operations on Tensors.
    # Now loss is a Tensor of shape (1,)
    # loss.item() gets the a scalar value held in the loss.
    loss = (y_pred - y).pow(2).sum()
    # Compute and print loss
    if epoch % 10 == 0:
        print ("Loss after epoch#{} = {}".format(epoch, loss.item()))

    # Use autograd to compute the backward pass. This call will compute the
    # gradient of loss with respect to all Tensors with requires_grad=True.
    # After this call w1.grad and w2.grad will be Tensors holding the gradient
    # of the loss with respect to w1 and w2 respectively.
    loss.backward()

    # Manually update weights using gradient descent. Wrap in torch.no_grad()
    # because weights have requires_grad=True, but we don't need to track this
    # in autograd.
    # An alternative way is to operate on weight.data and weight.grad.data.
    # Recall that tensor.data gives a tensor that shares the storage with
    # tensor, but doesn't track history.
    # You can also use torch.optim.SGD to achieve this.
    with torch.no_grad():
        w1 -= learning_rate * w1.grad
        w2 -= learning_rate * w2.grad

        # Manually zero the gradients after updating weights
        w1.grad.zero_()
        w2.grad.zero_()

Loss after epoch#10 = 2626682.5
Loss after epoch#20 = 284237.65625
Loss after epoch#30 = 96463.0859375
Loss after epoch#40 = 39380.3554688
Loss after epoch#50 = 17924.28125
Loss after epoch#60 = 8750.51855469
Loss after epoch#70 = 4483.93017578
Loss after epoch#80 = 2377.48120117
Loss after epoch#90 = 1295.20019531
Loss after epoch#100 = 721.10559082


# Implement custom Function with AutoGrad
In this section we implement our version of relu called myRelu
and we specify forward pass as well as backward pass

In [32]:
# -*- coding: utf-8 -*-
import torch


class MyReLU(torch.autograd.Function):
    """
    We can implement our own custom autograd Functions by subclassing
    torch.autograd.Function and implementing the forward and backward passes
    which operate on Tensors.
    """

    @staticmethod
    def forward(ctx, input):
        """
        In the forward pass we receive a Tensor containing the input and return
        a Tensor containing the output. ctx is a context object that can be used
        to stash information for backward computation. You can cache arbitrary
        objects for use in the backward pass using the ctx.save_for_backward method.
        """
        ctx.save_for_backward(input)
        return input.clamp(min=0)

    @staticmethod
    def backward(ctx, grad_output):
        """
        In the backward pass we receive a Tensor containing the gradient of the loss
        with respect to the output, and we need to compute the gradient of the loss
        with respect to the input.
        """
        input, = ctx.saved_tensors
        grad_input = grad_output.clone()
        grad_input[input < 0] = 0
        return grad_input

# Train model using Custom Function
Now we train a model using the custom implementation of RELU

In [33]:
dtype = torch.float
device = torch.device("cpu")
# device = torch.device("cuda:0") # Uncomment this to run on GPU

# N is batch size; D_in is input dimension;
# H is hidden dimension; D_out is output dimension.
N, D_in, H, D_out = 64, 1000, 100, 10

# Create random Tensors to hold input and outputs.
x = torch.randn(N, D_in, device=device, dtype=dtype)
y = torch.randn(N, D_out, device=device, dtype=dtype)

# Create random Tensors for weights.
w1 = torch.randn(D_in, H, device=device, dtype=dtype, requires_grad=True)
w2 = torch.randn(H, D_out, device=device, dtype=dtype, requires_grad=True)

learning_rate = 1e-6
for epoch in range(1, 101):
    # To apply our Function, we use Function.apply method. We alias this as 'relu'.
    relu = MyReLU.apply

    # Forward pass: compute predicted y using operations; we compute
    # ReLU using our custom autograd operation.
    y_pred = relu(x.mm(w1)).mm(w2)

    # Compute and print loss using operations on Tensors.
    # Now loss is a Tensor of shape (1,)
    # loss.item() gets the a scalar value held in the loss.
    loss = (y_pred - y).pow(2).sum()
    # Compute and print loss
    if epoch % 10 == 0:
        print ("Loss after epoch#{} = {}".format(epoch, loss.item()))

    # Use autograd to compute the backward pass.
    loss.backward()

    # Update weights using gradient descent
    with torch.no_grad():
        w1 -= learning_rate * w1.grad
        w2 -= learning_rate * w2.grad

        # Manually zero the gradients after updating weights
        w1.grad.zero_()
        w2.grad.zero_()

Loss after epoch#10 = 1696471.625
Loss after epoch#20 = 263291.0
Loss after epoch#30 = 86070.578125
Loss after epoch#40 = 33632.5039062
Loss after epoch#50 = 14504.90625
Loss after epoch#60 = 6679.05126953
Loss after epoch#70 = 3211.20874023
Loss after epoch#80 = 1596.90136719
Loss after epoch#90 = 815.346496582
Loss after epoch#100 = 425.159057617


## Use pytorch nn package to define NN
In this section we use pytorch.nn package but still perform manual backprop

In [35]:
# -*- coding: utf-8 -*-
import torch

# N is batch size; D_in is input dimension;
# H is hidden dimension; D_out is output dimension.
N, D_in, H, D_out = 64, 1000, 100, 10

# Create random Tensors to hold inputs and outputs
x = torch.randn(N, D_in)
y = torch.randn(N, D_out)

# Use the nn package to define our model as a sequence of layers. nn.Sequential
# is a Module which contains other Modules, and applies them in sequence to
# produce its output. Each Linear Module computes output from input using a
# linear function, and holds internal Tensors for its weight and bias.
model = torch.nn.Sequential(
    torch.nn.Linear(D_in, H),
    torch.nn.ReLU(),
    torch.nn.Linear(H, D_out),
)

# The nn package also contains definitions of popular loss functions; in this
# case we will use Mean Squared Error (MSE) as our loss function.
loss_fn = torch.nn.MSELoss(reduction='sum')

learning_rate = 1e-4
for epoch in range(1, 101):
    # Forward pass: compute predicted y by passing x to the model. Module objects
    # override the __call__ operator so you can call them like functions. When
    # doing so you pass a Tensor of input data to the Module and it produces
    # a Tensor of output data.
    y_pred = model(x)

    # Compute and print loss. We pass Tensors containing the predicted and true
    # values of y, and the loss function returns a Tensor containing the
    # loss.
    loss = loss_fn(y_pred, y)
    if epoch % 10 == 0:
        print ("Loss after epoch#{} = {}".format(epoch, loss.item()))

    # Zero the gradients before running the backward pass.
    model.zero_grad()

    # Backward pass: compute gradient of the loss with respect to all the learnable
    # parameters of the model. Internally, the parameters of each Module are stored
    # in Tensors with requires_grad=True, so this call will compute gradients for
    # all learnable parameters in the model.
    loss.backward()

    # Update the weights using gradient descent. Each parameter is a Tensor, so
    # we can access its gradients like we did before.
    with torch.no_grad():
        for param in model.parameters():
            param -= learning_rate * param.grad

Loss after epoch#10 = 330.044769287
Loss after epoch#20 = 193.611541748
Loss after epoch#30 = 110.871696472
Loss after epoch#40 = 61.8626441956
Loss after epoch#50 = 34.657333374
Loss after epoch#60 = 19.8393688202
Loss after epoch#70 = 11.6505479813
Loss after epoch#80 = 7.00678491592
Loss after epoch#90 = 4.30923891068
Loss after epoch#100 = 2.70535635948


# Use pytorch.optim package
We use AdamOptimizer from the optim package to train our simple NN

In [36]:
# -*- coding: utf-8 -*-
import torch

# N is batch size; D_in is input dimension;
# H is hidden dimension; D_out is output dimension.
N, D_in, H, D_out = 64, 1000, 100, 10

# Create random Tensors to hold inputs and outputs
x = torch.randn(N, D_in)
y = torch.randn(N, D_out)

# Use the nn package to define our model and loss function.
model = torch.nn.Sequential(
    torch.nn.Linear(D_in, H),
    torch.nn.ReLU(),
    torch.nn.Linear(H, D_out),
)
loss_fn = torch.nn.MSELoss(reduction='sum')

# Use the optim package to define an Optimizer that will update the weights of
# the model for us. Here we will use Adam; the optim package contains many other
# optimization algoriths. The first argument to the Adam constructor tells the
# optimizer which Tensors it should update.
learning_rate = 1e-4
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
for epoch in range(1, 101):
    # Forward pass: compute predicted y by passing x to the model.
    y_pred = model(x)

    # Compute and print loss.
    loss = loss_fn(y_pred, y)
    if epoch % 10 == 0:
        print ("Loss after epoch#{} = {}".format(epoch, loss.item()))

    # Before the backward pass, use the optimizer object to zero all of the
    # gradients for the variables it will update (which are the learnable
    # weights of the model). This is because by default, gradients are
    # accumulated in buffers( i.e, not overwritten) whenever .backward()
    # is called. Checkout docs of torch.autograd.backward for more details.
    optimizer.zero_grad()

    # Backward pass: compute gradient of the loss with respect to model
    # parameters
    loss.backward()

    # Calling the step function on an Optimizer makes an update to its
    # parameters
    optimizer.step()

Loss after epoch#10 = 581.626831055
Loss after epoch#20 = 463.609008789
Loss after epoch#30 = 373.10748291
Loss after epoch#40 = 300.95022583
Loss after epoch#50 = 241.537628174
Loss after epoch#60 = 192.123901367
Loss after epoch#70 = 150.791213989
Loss after epoch#80 = 116.273269653
Loss after epoch#90 = 87.9394683838
Loss after epoch#100 = 65.2383651733


## Define custom model using nn.Module
In this section we define a custom 2 layer NN and use this class to train our model

In [37]:
# -*- coding: utf-8 -*-
import torch


class TwoLayerNet(torch.nn.Module):
    def __init__(self, D_in, H, D_out):
        """
        In the constructor we instantiate two nn.Linear modules and assign them as
        member variables.
        """
        super(TwoLayerNet, self).__init__()
        self.linear1 = torch.nn.Linear(D_in, H)
        self.linear2 = torch.nn.Linear(H, D_out)

    def forward(self, x):
        """
        In the forward function we accept a Tensor of input data and we must return
        a Tensor of output data. We can use Modules defined in the constructor as
        well as arbitrary operators on Tensors.
        """
        h_relu = self.linear1(x).clamp(min=0)
        y_pred = self.linear2(h_relu)
        return y_pred


# N is batch size; D_in is input dimension;
# H is hidden dimension; D_out is output dimension.
N, D_in, H, D_out = 64, 1000, 100, 10

# Create random Tensors to hold inputs and outputs
x = torch.randn(N, D_in)
y = torch.randn(N, D_out)

# Construct our model by instantiating the class defined above
model = TwoLayerNet(D_in, H, D_out)

# Construct our loss function and an Optimizer. The call to model.parameters()
# in the SGD constructor will contain the learnable parameters of the two
# nn.Linear modules which are members of the model.
criterion = torch.nn.MSELoss(reduction='sum')
optimizer = torch.optim.SGD(model.parameters(), lr=1e-4)
for epoch in range(1, 101):
    # Forward pass: Compute predicted y by passing x to the model
    y_pred = model(x)

    # Compute and print loss
    loss = criterion(y_pred, y)
    if epoch % 10 == 0:
        print ("Loss after epoch#{} = {}".format(epoch, loss.item()))

    # Zero gradients, perform a backward pass, and update the weights.
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

Loss after epoch#10 = 413.988739014
Loss after epoch#20 = 240.924194336
Loss after epoch#30 = 135.251693726
Loss after epoch#40 = 73.4742279053
Loss after epoch#50 = 39.767250061
Loss after epoch#60 = 21.9052352905
Loss after epoch#70 = 12.3572177887
Loss after epoch#80 = 7.16591978073
Loss after epoch#90 = 4.27396297455
Loss after epoch#100 = 2.6149096489


## DynamicNet - An unusual model
To illustrate dynamic computation graphs, we implement a network where forward propogation will reuse
same weights for oneof the hidden layer that is evaluated 0, 1, 2 or 3 times based on a random variable.
This model is defined to illustrate dynamic computation graphs

In [38]:
# -*- coding: utf-8 -*-
import random
import torch


class DynamicNet(torch.nn.Module):
    def __init__(self, D_in, H, D_out):
        """
        In the constructor we construct three nn.Linear instances that we will use
        in the forward pass.
        """
        super(DynamicNet, self).__init__()
        self.input_linear = torch.nn.Linear(D_in, H)
        self.middle_linear = torch.nn.Linear(H, H)
        self.output_linear = torch.nn.Linear(H, D_out)

    def forward(self, x):
        """
        For the forward pass of the model, we randomly choose either 0, 1, 2, or 3
        and reuse the middle_linear Module that many times to compute hidden layer
        representations.

        Since each forward pass builds a dynamic computation graph, we can use normal
        Python control-flow operators like loops or conditional statements when
        defining the forward pass of the model.

        Here we also see that it is perfectly safe to reuse the same Module many
        times when defining a computational graph. This is a big improvement from Lua
        Torch, where each Module could be used only once.
        """
        h_relu = self.input_linear(x).clamp(min=0)
        for _ in range(random.randint(0, 3)):
            h_relu = self.middle_linear(h_relu).clamp(min=0)
        y_pred = self.output_linear(h_relu)
        return y_pred


# N is batch size; D_in is input dimension;
# H is hidden dimension; D_out is output dimension.
N, D_in, H, D_out = 64, 1000, 100, 10

# Create random Tensors to hold inputs and outputs
x = torch.randn(N, D_in)
y = torch.randn(N, D_out)

# Construct our model by instantiating the class defined above
model = DynamicNet(D_in, H, D_out)

# Construct our loss function and an Optimizer. Training this strange model with
# vanilla stochastic gradient descent is tough, so we use momentum
criterion = torch.nn.MSELoss(reduction='sum')
optimizer = torch.optim.SGD(model.parameters(), lr=1e-4, momentum=0.9)
for epoch in range(1, 101):
    # Forward pass: Compute predicted y by passing x to the model
    y_pred = model(x)

    # Compute and print loss
    loss = criterion(y_pred, y)
    if epoch % 10 == 0:
        print ("Loss after epoch#{} = {}".format(epoch, loss.item()))


    # Zero gradients, perform a backward pass, and update the weights.
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()


Loss after epoch#10 = 236.231506348
Loss after epoch#20 = 349.229034424
Loss after epoch#30 = 159.830245972
Loss after epoch#40 = 126.480506897
Loss after epoch#50 = 171.989425659
Loss after epoch#60 = 20.1855106354
Loss after epoch#70 = 25.2583751678
Loss after epoch#80 = 41.6708030701
Loss after epoch#90 = 14.4199466705
Loss after epoch#100 = 30.158908844


In [45]:
# print model parameters
for p in model.parameters():
    if p.requires_grad:
         print(p.name, p.data)

(None, tensor([[-2.7946e-02,  1.3537e-03, -4.6445e-02,  ...,  8.6383e-03,
         -1.4318e-02, -3.9209e-02],
        [ 2.5804e-03,  1.2038e-02, -7.3610e-03,  ..., -1.4908e-02,
         -2.9497e-02,  1.1369e-03],
        [-4.1324e-03, -2.1633e-03, -2.8950e-02,  ..., -5.5250e-03,
          1.2308e-02,  2.9307e-02],
        ...,
        [-2.1104e-02,  2.9639e-02,  1.4718e-02,  ..., -2.0050e-02,
         -3.0024e-02, -1.1952e-02],
        [-3.6979e-05,  1.2809e-02,  5.3615e-03,  ..., -2.5803e-02,
         -2.7878e-02, -1.4689e-02],
        [ 1.2043e-02, -7.2605e-04,  2.3104e-02,  ...,  1.9859e-02,
          1.3871e-02, -1.4269e-02]]))
(None, tensor([-0.0130,  0.0148, -0.0072, -0.0361,  0.0065, -0.0196,  0.0245,  0.0007,
         0.0117, -0.0269,  0.0207, -0.0015,  0.0116,  0.0132, -0.0104, -0.0074,
        -0.0221, -0.0253, -0.0350,  0.0249,  0.0035, -0.0145,  0.0139,  0.0209,
        -0.0150, -0.0065, -0.0078, -0.0105,  0.0310, -0.0232, -0.0065,  0.0135,
        -0.0108, -0.0028, -0.0142